# 🚤 Day 2: Dive! (aka Yellow Submarine)

*Now, you need to figure out how to pilot this thing.*

## Part 1

Seems like we have a submarine that can take a series of commands, but it all comes down to:
- ➡ We can move the submarine in the **horizontal direction** (only positive displacements)
- ⬆⬇ We can move the submarine in the **vertical direction** (positive and negative displacements)

We'll go over how to know where and how much to move, but first let's create our submarine.

### ✨ Building and testing a Submarine class

Our little submarine will be used for basic traveling, so we only really need to **know its coordinates** and **be able to update them by moving the submarine.** We'll keep things and simple and think about movement in the horizontal component and in the vertical component. We'll worry about whether to move up or down later on.

So, let's create a [dataclass](https://docs.python.org/3/library/dataclasses.html) for our vehicle. It will have a `horizontal_position` and a `depth`, which will be initialized as 0 by default, and it will also have two methods: one to update the horizontal coordinates (`move_horizontal`) and another for the vertical coordinates (`move_vertical`).

To make sure the submarine is understanding our commands, we will set a simple path (1 down, 2 forward, 3 up) and see if it lands on the right coordinates. 

In [29]:
from dataclasses import dataclass

@dataclass
class Submarine:
    """Represents a submarine moving through the sea"""
    horizontal_position: int = 0
    depth: int = 0
        
    def move_vertical(self, distance: int) -> None:
        """Move the submarine downwards by a distance"""
        self.depth += distance
        
    def move_horizontal(self, distance: int) -> None:
        """Move the submarine forward by a distance"""
        self.horizontal_position +=  distance
        
    def print_position(self) -> None:
        """Prints the current position of the submarine"""
        print(f"I'm currently at depth {self.depth}, horizontal {self.horizontal_position}")
        
# Test the submarine class, should be at (2, -2)
submarine = Submarine()
submarine.move_vertical(1)
submarine.move_horizontal(2)
submarine.move_vertical(-3)
submarine.print_position()

# Bonus: Dataclasses are cool and we don't really need the print_position() method
submarine

I'm currently at depth -2, horizontal 2


Submarine(horizontal_position=2, depth=-2)

### 🔍 Extracting information from the input .txt file

The structure of our input file is quite easy to work with. We'll need to know the **direction** of movement (up, down or forward) and the **distance** by which the submarine will move. Each line follows the same structure: 

"`direction` `distance`". 

In other words, our variables are separated by a blank space, which we can use to [split](https://docs.python.org/3/library/stdtypes.html#str.split) the string. Also, we can use the [rstrip](https://www.w3schools.com/python/ref_string_rstrip.asp) method to remove the trailing newline character ("\n") at the end of the string. (And don't forget to make distance a numerical value to make things easier)

Let's combine these ideas to create a function that extracts the direction and distance data from the input lines!

In [41]:
def get_info_from_file_string(line: str):
    """Extracts the direction and distance value from string"""
    # Get the direction and distance from the txt file
    direction, distance = line.rstrip("\n").split(" ")
    distance = int(distance)
    
    return direction, distance

### 🧩 Solving the puzzle

Now that we have all the main pieces to solve the puzzle, let's put them together. Here is our planned workflow:

- Open the file and read each line:
    - Extract the needed info
    - If the direction is **"forward"**, we update the submarine's **horizontal position** and **move to the next line**;
    - **Otherwise**, we update the submarine's **vertical position**:
        - If the direction is **"up"**, we need to make the displacement value **negative**
        - Update the submarine's **vertical position** and **move to the next line**

In [44]:
# Let's dive into the input file with our Submarine
submarine = Submarine()

with open("day2/input.txt") as fileobject:
    for line in fileobject:
        # Get the direction and distance from the txt file
        direction, distance = get_info_from_file_string(line)
        
        # Decide how to move the submarine
        # If direction is forward, go for horizontal movement
        if direction == "forward":
            submarine.move_horizontal(distance)
            continue
            
        # Otherwise, go for vertical movement and assign the correct direction
        # Up corresponds to negative movement, down corresponds to positive
        if direction == "up":
            distance = - distance
            
        submarine.move_vertical(distance)
        
submarine.print_position()
# Print answer
print(submarine.depth * submarine.horizontal_position)

I'm currently at depth 1090, horizontal 1868
2036120


## Part 2

In part two our commands get a bit more sophisticated but we can still think about two main components:

- ⏩ We can **move the submarine** (possibly both coordinates at once)
- 🔄 We can **update the submarine's aim**

### ✨ Building and testing a Submarine subclass with aim

The yellow submarine needs an upgrade. It still moves up, down and forward, but now it has a new component: the aim, which controls the vertical movement. So, we can **extend our previous class** to take advantage of the code we wrote to **move the boat**, and we will **implement the aim component and how it behaves into a new subclass**.

We need to add a `aim` attribute to store the value of this component. In addition, we should create a new method to move our submarine, as **the new rules for movement consider both coordinates**. So, we will write a `move_submarine` method that **updates the horizontal coordinate and subsequently updates the vertical coordinate** based on the current aim value.

In [35]:
@dataclass
class SubmarineWithAim(Submarine):
    """An extension of the submarine object with aim"""
    aim: int = 0
        
    def add_aim(self, value) -> None:
        """Updates the aim based on a new value"""
        self.aim += value
        
    def move_submarine(self, distance):
        """Moves the submarine (horizontal + vertical with aim)"""
        # Move the horizontal component
        self.move_horizontal(distance)
        # Get the vertical component and move it
        vertical_movement = self.get_distance_with_aim(distance)
        self.move_vertical(vertical_movement)
        
    def get_distance_with_aim(self, distance) -> int:
        """Returns the new vertical distance value based on the current aim"""
        return distance * self.aim
        
# Test the subclass, should be at (10, 50)
aim_submarine = SubmarineWithAim()
aim_submarine.add_aim(5)
aim_submarine.move_submarine(10)
aim_submarine.print_position()

I'm currently at depth 50, horizontal 10


### 🧩 Solving the puzzle

Our previous workflow can be adapted to the new rules:

- Open the file and read each line:
    - Extract the needed info
    - If the direction is **"forward"**, we **move the submarine** (the move method takes care of it) and **move to the next line**;
    - **Otherwise**, we update the submarine's **aim**:
        - If the direction is **"up"**, we need to make the aim value **negative**
        - Update the submarine's **aim** and **move to the next line**

In [45]:
# Let's dive into the input file with our special SubmarineWithAim
aim_submarine = SubmarineWithAim()

with open("day2/input.txt") as fileobject:
    for line in fileobject:
        # Get the direction and distance from the txt file
        direction, distance = get_info_from_file_string(line)
        
        # Move the submarine
        if direction == "forward":
            aim_submarine.move_submarine(distance)
            continue
            
        # Update the aim
        if direction == "up":
            distance = - distance
            
        aim_submarine.add_aim(distance)
        
aim_submarine.print_position()
# Print answer
print(aim_submarine.depth * aim_submarine.horizontal_position)

I'm currently at depth 1078987, horizontal 1868
2015547716
